In [19]:
import nltk
import numpy as np
import string
import random
import requests
import re
import warnings
warnings.filterwarnings('ignore')

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
nltk.download('popular', quiet = True)

True

In [22]:
response = requests.get('https://en.wikipedia.org/wiki/Chatbot')
raw_html = response.text

In [23]:
import bs4 as bs
corpus_html = bs.BeautifulSoup(raw_html)
corpus_paras = corpus_html.find_all('p')
corpus_text = ''

for para in corpus_paras:
  corpus_text += para.text

corpus_text = corpus_text.lower()

In [24]:
corpus_text

'a chatbot or chatterbot is a software application used to conduct an on-line chat conversation via text or text-to-speech, in lieu of providing direct contact with a live human agent.[1][2] a chatbot is a type of software that can help customers by automating conversations and interact with them through messaging platforms.[3] designed to convincingly simulate the way a human would behave as a conversational partner, chatbot systems typically require continuous tuning and testing, and many in production remain unable to adequately converse, while none of them can pass the standard turing test.[4] the term "chatterbot" was originally coined by michael mauldin (creator of the first verbot) in 1994 to describe these conversational programs.[5]\nchatbots are used in dialog systems for various purposes including customer service, request routing, or information gathering. while some chatbot applications use extensive word-classification processes, natural-language processors, and sophistic

In [62]:
corpus_text = re.sub(r'\[^0-9]', ' ', corpus_text)
corpus_text = re.sub(r'\s+' , ' ', corpus_text)
corpus_text = re.sub(r"\[[\d]+]", ' ',corpus_text)

In [63]:
corpus_sentences = nltk.sent_tokenize(corpus_text)
corpus_words = nltk.word_tokenize(corpus_text)

In [64]:
corpus_sentences

['a chatbot or chatterbot is a software application used to conduct an on line chat conversation via text or text to speech in lieu of providing direct contact with a live human agent 1 2 a chatbot is a type of software that can help customers by automating conversations and interact with them through messaging platforms 3 designed to convincingly simulate the way a human would behave as a conversational partner chatbot systems typically require continuous tuning and testing and many in production remain unable to adequately converse while none of them can pass the standard turing test 4 the term chatterbot was originally coined by michael mauldin creator of the first verbot in 1994 to describe these conversational programs 5 chatbots are used in dialog systems for various purposes including customer service request routing or information gathering while some chatbot applications use extensive word classification processes natural language processors and sophisticated ai others simply 

In [61]:
corpus_words

['a',
 'chatbot',
 'or',
 'chatterbot',
 'is',
 'a',
 'software',
 'application',
 'used',
 'to',
 'conduct',
 'an',
 'on-line',
 'chat',
 'conversation',
 'via',
 'text',
 'or',
 'text-to-speech',
 ',',
 'in',
 'lieu',
 'of',
 'providing',
 'direct',
 'contact',
 'with',
 'a',
 'live',
 'human',
 'agent',
 '.',
 '[',
 '1',
 ']',
 '[',
 '2',
 ']',
 'a',
 'chatbot',
 'is',
 'a',
 'type',
 'of',
 'software',
 'that',
 'can',
 'help',
 'customers',
 'by',
 'automating',
 'conversations',
 'and',
 'interact',
 'with',
 'them',
 'through',
 'messaging',
 'platforms',
 '.',
 '[',
 '3',
 ']',
 'designed',
 'to',
 'convincingly',
 'simulate',
 'the',
 'way',
 'a',
 'human',
 'would',
 'behave',
 'as',
 'a',
 'conversational',
 'partner',
 ',',
 'chatbot',
 'systems',
 'typically',
 'require',
 'continuous',
 'tuning',
 'and',
 'testing',
 ',',
 'and',
 'many',
 'in',
 'production',
 'remain',
 'unable',
 'to',
 'adequately',
 'converse',
 ',',
 'while',
 'none',
 'of',
 'them',
 'can',
 'pass'

In [26]:
greeting_inputs = ("hey", "hi", "good morning", "morning", "good evening", "good afternoon", "hello")
greeting_responses = ("hey", "how are you", "how you doing", "hello", "what do you do")

In [27]:
def greet_response(greeting):
  for token in greeting.split():
    if token in greeting_inputs:
      return random.choice(greeting_responses)

In [28]:
lemmatize = nltk.stem.WordNetLemmatizer()

def lemmatize_corpus(tokens):
  return (lemmatize.lemmatize(token) for token in tokens)

punct_removal_dict = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_preprocessed_text(document):
  return lemmatize_corpus(nltk.word_tokenize(document.lower().translate(punct_removal_dict)))

In [29]:
def response(user_response):
    robo_response=''
    corpus_sentences.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer= get_preprocessed_text, stop_words='english')
    tfidf = TfidfVec.fit_transform(corpus_sentences)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+corpus_sentences[idx]
        return robo_response

In [60]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greet_response(user_response)!=None):
                print("ROBO: "+greet_response(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                corpus_sentences.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
hey
ROBO: what do you do
nothing
ROBO: I am sorry! I don't understand you
can you say me about chatbots
ROBO: but once a particular program is unmasked, once its inner workings are explained ... its magic crumbles away; it stands revealed as a mere collection of procedures ... the observer says to himself "i could have written that".
chatbots
ROBO: [45][46] chatbots are also appearing in the healthcare industry.
chatbot
ROBO: [75] chatbot jobs chatbot developers create, debug, and maintain applications that automate customer services or other communication processes.
bye
ROBO: Bye! take care..
